# Partie 3 : Machine Learning pour prédire la survie des patients

### Dans une dernière étape, on s’intéresse à la prédiction de la survie globale, variable binaire indiquant si le patient est vivant ou décédé après l'étude. Les données proviennet de cette étude : https://doi.org/10.1038/s41586-024-08167-5.

- On entraine des modèles pour les différents types de cancers (poumon, colorectal, sein, prostate et pancréas). Les variables explicatives retenues sont génomiques (TMB, fraction du génome altéré, score MSI) et cliniques (âge, stade tumoral,sexe, pureté tumorale).

- Trois algorithmes sont comparés : régression logistique, RandomForest et KNN. Les performances ont été évaluées avec l’AUC et la balanced accuracy.

- Les résultats montrent que la régression logistique et RandomForest ont des performances comparables, généralement supérieures à KNN, avec des AUC autour de 0.70. Ces valeurs indiquent un pouvoir prédictif modéré, suggérant que les variables utilisées apportent de l’information utile mais ne se suffisent pas. Les différences observées entre types de cancer reflètent alors de la diversité biologique et clinique de ces pathologies.

Traitement des données, choix des variables.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
import pandas as pd

df = pd.read_csv("msk_chord_2024_clinical_data.tsv.gz", sep="\t")

# Variable cible
y = df["Overall Survival Status"].apply(lambda x: 1 if "1:DECEASED" in str(x).upper() else 0)

# Séparer quanti et quali
num_features = ["TMB (nonsynonymous)", "Fraction Genome Altered", "MSI Score", "Current Age", "Tumor Purity"]

cat_features = ["Sex", "Stage (Highest Recorded)"]

# Cancers étudiés par les chercheurs
cancer_types = ["Non-Small Cell Lung Cancer", "Colorectal Cancer", "Breast Cancer", 
                "Prostate Cancer", "Pancreatic Cancer"]

In [2]:
# Préprocesseurs
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ])

In [3]:
# Modèles
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced"),
    "KNN": KNeighborsClassifier(n_neighbors=5)  # KNN pas adapté si données avec trop de bruit, regardons ce qu'il se passe
}

Entrainement, validation et test des modèles.

In [4]:
results = []
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
# Boucle par type de cancer
for cancer in cancer_types:
    df_cancer = df[df["Cancer Type"] == cancer].copy()
    
    # Features et cible
    X = df_cancer[num_features + cat_features]
    y = df_cancer["Overall Survival Status"].apply(lambda x: 1 if "1:DECEASED" in str(x).upper() else 0)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    for name, model in models.items():
        # Pipeline avec SMOTE
        clf = ImbPipeline(steps=[
            ("preprocessor", preprocessor),
            ("smote", SMOTE(random_state=42)),
            ("classifier", model)
        ])
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)[:, 1]
        
        auc = roc_auc_score(y_test, y_prob)
        balac = balanced_accuracy_score(y_test, y_pred)
        
        results.append({
            "Cancer": cancer,
            "Modèle": name,
            "AUC": round(auc, 3),
            "Balanced Accuracy": round(balac, 3)
        })

# Résumé des résultats dans un tableau
results_df = pd.DataFrame(results)
print(results_df)



                        Cancer               Modèle    AUC  Balanced Accuracy
0   Non-Small Cell Lung Cancer  Logistic Regression  0.708              0.668
1   Non-Small Cell Lung Cancer        Random Forest  0.705              0.646
2   Non-Small Cell Lung Cancer                  KNN  0.666              0.626
3            Colorectal Cancer  Logistic Regression  0.721              0.688
4            Colorectal Cancer        Random Forest  0.707              0.662
5            Colorectal Cancer                  KNN  0.653              0.620
6                Breast Cancer  Logistic Regression  0.670              0.629
7                Breast Cancer        Random Forest  0.683              0.624
8                Breast Cancer                  KNN  0.595              0.566
9              Prostate Cancer  Logistic Regression  0.705              0.656
10             Prostate Cancer        Random Forest  0.711              0.650
11             Prostate Cancer                  KNN  0.625      